<a href="https://colab.research.google.com/github/DhvaniDoshi/Fake-News-Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Detection
 

In [34]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [36]:
from google.colab import drive
drive.mount('/content/drive')
df_fake = pd.read_csv("/content/drive/MyDrive/Fake.csv")
df_true = pd.read_csv("/content/drive/MyDrive/True.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
df_fake.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [38]:
df_true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [39]:
df_fake["class"]=0
df_true["class"]=1

In [40]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [41]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
  df_fake.drop([i],axis=0,inplace=True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
  df_true.drop([i],axis=0,inplace=True)  

In [42]:
df_manual_testing=pd.concat([df_fake_manual_testing,df_true_manual_testing],axis=0)
df_manual_testing.to_csv("/content/drive/MyDrive/manual_testing.csv")


In [43]:
df_marge=pd.concat([df_fake,df_true],axis=0)  # Two dataframe into single frame using marge
df_marge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [44]:
df = df_marge.drop(["title","subject","date"],axis=1) #dataframe
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [45]:
df = df.sample(frac=1) #dataset

In [46]:
df.head(10)

,text,class
23184,21st Century Wire says Amid great mainstream ...,0
10599,Laura Loomer of Rebel Media rushed the stage a...,0
4012,WASHINGTON (Reuters) - U.S. President Donald ...,1
18964,YANGON (Reuters) - Myanmar is committing crime...,1
12471,BRUSSELS (Reuters) - NATO allies extended Jens...,1
17536,LONDON (Reuters) - British foreign minister Bo...,1
22898,21st Century Wire says Investigative reporter ...,0
17913,San Antonio Spurs coach Gregg Popovich is a to...,0
18455,Karen Handel won a hard fought Congressional s...,0
12552,You just can t make this up! CNN fires black D...,0


In [47]:
# for checking any null value
df.isnull().sum()

text     0
class    0
dtype: int64

In [48]:
# for removing unnecessary text
def word_drop(text):
  text = text.lower()
  text = re.sub('\[.*?\]'," ",text)
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://\S+|www\.\S+','',text)
  text = re.sub('<.*?>+','',text)
  text = re.sub('[%s]'% re.escape(string.punctuation),'',text)
  text = re.sub('\n','',text)
  text = re.sub('\w*\d\w*','',text)
  return text


In [49]:
df["text"] = df["text"].apply(word_drop)

In [50]:
df.head(10) # for showing changes in dataframe 

,text,class
23184,century wire says amid great mainstream medi...,0
10599,laura loomer of rebel media rushed the stage a...,0
4012,washington reuters u s president donald ...,1
18964,yangon reuters myanmar is committing crime...,1
12471,brussels reuters nato allies extended jens...,1
17536,london reuters british foreign minister bo...,1
22898,century wire says investigative reporter robe...,0
17913,san antonio spurs coach gregg popovich is a to...,0
18455,karen handel won a hard fought congressional s...,0
12552,you just can t make this up cnn fires black d...,0


In [51]:
# Define dependent and independent variable
x = df["text"]
y = df["class"] # insert new coloum called class insert new category true=1 and fake= 0 


In [52]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .25)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization =  TfidfVectorizer() # Vectorizer
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

###Logistic Regression



In [64]:
# for import logistic regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [65]:
LR.score(xv_test,y_test)

0.9850267379679144

In [66]:
pred_LR = LR.predict(xv_test)

In [67]:
print(classification_report(y_test,pred_LR)) # for showing Logistic Regression Model

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5821
           1       0.98      0.99      0.98      5399

    accuracy                           0.99     11220
   macro avg       0.98      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Decision Tree Classification

In [70]:
# Decision Tree Classification
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [71]:
DT.score(xv_test,y_test)

0.9959001782531194

In [72]:
pred_DT = DT.predict(xv_test)

In [73]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5821
           1       1.00      1.00      1.00      5399

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Gradient Boosting Classifier

In [74]:
from sklearn.ensemble import GradientBoostingClassifier # tree base algorithm ,it used for classification & Regression

In [76]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [78]:
GBC.score(xv_test,y_test)

0.9954545454545455

In [79]:
pred_GBC = GBC.predict(xv_test)

In [80]:
print(classification_report(y_test,pred_GBC))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5821
           1       0.99      1.00      1.00      5399

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



###Random Forest Classifier

In [81]:
from sklearn.ensemble import RandomForestClassifier

In [82]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [83]:
RFC.score(xv_test,y_test) # for checking Accuracy

0.9893939393939394

In [84]:
pred_RFC = RFC.predict(xv_test)

In [85]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5821
           1       0.99      0.99      0.99      5399

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### Manual Testing

In [1]:
def output_label(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "Not A Fake News"

def manual_testing(news):
     testing_news = {"text":[news]}
     new_def_test = pd.DataFrame(testing_news)
     new_def_test["text"] = new_def_test["text"].apply(word_drop)
     new_x_test = new_def_test["text"]
     new_xv_test = vectorization.transform(new_x_test)
     pred_LR = LR.predict(new_xv_test)
     pred_DT = DT.predict(new_xv_test)
     pred_GBC = GBC.predict(new_xv_test)
     pred_RFC = RFC.predict(new_xv_test)
     return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                                output_lable(pred_DT[0]),
                                                                                                                output_lable(pred_GBC[0]),
                                                                                                                output_lable(pred_RFC[0])))

In [87]:
import pandas as pd 
news = str(input())
manual_testing(news)

Switzerland Continue this story at the Mail OnlineREAD MORE IRAN NEWS AT: 21st Century Wire Iran Files


NameError: ignored